# Лабораторна робота No1
Наука про дані: підготовчий етап
ФБ-21 Худоба Арсен
Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції

1. Для кожної із адміністративних одиниць України завантажити текстові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [7]:
import os
import pandas as pd
import urllib.request
from datetime import datetime
import shutil

In [11]:
directory = 'csv_files'
if os.path.exists(directory):
        shutil.rmtree(directory)
        
def get_data(i):
    os.makedirs(directory, exist_ok=True)
    
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)

    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f'vhi_id_{i}_{date_time}.csv'

    filepath = os.path.join(directory, filename)
    with open(filepath, 'wb') as out:
        out.write(vhi_url.read())

    print(f"VHI for area {i} is downloaded and saved as {filename} in {directory} directory.")

for i in range(1, 28):
    get_data(i)

VHI for area 1 is downloaded and saved as vhi_id_1_2024-03-07_21-05-17.csv in csv_files directory.
VHI for area 2 is downloaded and saved as vhi_id_2_2024-03-07_21-05-17.csv in csv_files directory.
VHI for area 3 is downloaded and saved as vhi_id_3_2024-03-07_21-05-18.csv in csv_files directory.
VHI for area 4 is downloaded and saved as vhi_id_4_2024-03-07_21-05-19.csv in csv_files directory.
VHI for area 5 is downloaded and saved as vhi_id_5_2024-03-07_21-05-20.csv in csv_files directory.
VHI for area 6 is downloaded and saved as vhi_id_6_2024-03-07_21-05-21.csv in csv_files directory.
VHI for area 7 is downloaded and saved as vhi_id_7_2024-03-07_21-05-22.csv in csv_files directory.
VHI for area 8 is downloaded and saved as vhi_id_8_2024-03-07_21-05-23.csv in csv_files directory.
VHI for area 9 is downloaded and saved as vhi_id_9_2024-03-07_21-05-23.csv in csv_files directory.
VHI for area 10 is downloaded and saved as vhi_id_10_2024-03-07_21-05-24.csv in csv_files directory.
VHI for 

2. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [26]:
def read(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir(directory)
    result_df = pd.DataFrame()
    for i in range(len(files)):
        full_path = os.path.join(directory, files[i])
        df = pd.read_csv(full_path, header=1, names=headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['area'] = i + 1
        df['Year'] = df['Year'].str.replace('<tt><pre>', '')
        df = df[~df['Year'].str.contains('</pre></tt>')]
        df.drop('empty', axis=1, inplace=True)
        result_df = pd.concat([result_df, df])
    return result_df
df = read(directory)
print(df)


      Year  Week    SMN     SMT    VCI    TCI    VHI  area
0     1982   1.0  0.059  258.24  51.11  48.78  49.95     1
1     1982   2.0  0.063  261.53  55.89  38.20  47.04     1
2     1982   3.0  0.063  263.45  57.30  32.69  44.99     1
3     1982   4.0  0.061  265.10  53.96  28.62  41.29     1
4     1982   5.0  0.058  266.42  46.87  28.57  37.72     1
...    ...   ...    ...     ...    ...    ...    ...   ...
2188  2024   5.0  0.101  275.13  58.55   8.77  33.66    27
2189  2024   6.0  0.110  276.43  60.83   9.08  34.96    27
2190  2024   7.0  0.120  277.97  61.35  10.45  35.90    27
2191  2024   8.0  0.130  279.15  60.18  13.87  37.02    27
2192  2024   9.0  0.142  280.64  59.61  16.63  38.12    27

[57861 rows x 8 columns]


3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на потрібні.

In [27]:
def changeID(df):
    cities =  {1: "Cherkasy", 2: "Chernihiv", 3: "Chernivtsi", 4: "Crimea", 5: "Dnipropetrovs'k", 6: "Donets'k", 7: "Ivano-Frankivs'k", 8: "Kharkiv", 9: "Kherson", 10: "Khmel'nyts'kyy", 11: "Kiev", 12: "Kiev City", 13: "Kirovohrad", 14: "Luhans'k", 15: "L'viv", 16: "Mykolayiv", 17: "Odessa", 18: "Poltava", 19: "Rivne", 20: "Sevastopol'", 21: "Sumy", 22: "Ternopil'", 23: "Transcarpathia", 24: "Vinnytsya", 25: "Volyn", 26: "Zaporizhzhya", 27: "Zhytomyr"}
    df['area'] = df['area'].replace(cities)
    
    return df

In [28]:
df = changeID(read(directory))
print(df)

      Year  Week    SMN     SMT    VCI    TCI    VHI      area
0     1982   1.0  0.059  258.24  51.11  48.78  49.95  Cherkasy
1     1982   2.0  0.063  261.53  55.89  38.20  47.04  Cherkasy
2     1982   3.0  0.063  263.45  57.30  32.69  44.99  Cherkasy
3     1982   4.0  0.061  265.10  53.96  28.62  41.29  Cherkasy
4     1982   5.0  0.058  266.42  46.87  28.57  37.72  Cherkasy
...    ...   ...    ...     ...    ...    ...    ...       ...
2188  2024   5.0  0.101  275.13  58.55   8.77  33.66  Zhytomyr
2189  2024   6.0  0.110  276.43  60.83   9.08  34.96  Zhytomyr
2190  2024   7.0  0.120  277.97  61.35  10.45  35.90  Zhytomyr
2191  2024   8.0  0.130  279.15  60.18  13.87  37.02  Zhytomyr
2192  2024   9.0  0.142  280.64  59.61  16.63  38.12  Zhytomyr

[57861 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
o Ряд VHI для області за рік, пошук екстремумів (min та max);

In [29]:
def max_extr(df, area, year):
    max_vhi = df[(df['area'] == area) & (df['Year'] == year)]['VHI'].max()
    
    return max_vhi

max_vhi = max_extr(df, "Kiev", '2022')
print('max:', max_vhi)

def min_extr(df, area, year):
    min_vhi = df[(df['area'] == area) & (df['Year'] == year)]['VHI'].min()
    
    return min_vhi

min_vhi = min_extr(df, "Kiev", '2022')
print('min:', min_vhi)

max: 72.32
min: 36.12


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
o Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка області;

In [58]:
def dry(df, area, percent):
    df_area = df[df["area"] == area]
    df_drought = df[(df.VHI <= percent)]["Year"].unique()
    df_drought.sort()
    return df_drought


print(dry(df, "Kiev", 15))

['1984' '1986' '1993' '1994' '1999' '2000' '2003' '2007' '2012']


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
o Аналогічно для помірних посух

In [63]:
def dry(df, area, morethan, lessthan):
    df_area = df[df["area"] == area]
    df_drought = df[(df.VHI >= morethan) & (df.VHI <= lessthan)]["Year"].unique()
    df_drought.sort()
    return df_drought


print(dry(df, "Kiev", 0, 15))

['1984' '1986' '1993' '1994' '1999' '2000' '2003' '2007' '2012']
